In [ ]:
import pandas as pd
from fastai.tabular.all import *
import numpy as np

In [ ]:
set_seed(42)

TRAIN = '../input/ubiquant-trainfeather/train.feather'
EXAMPLE_TEST = '../input/ubiquant-market-prediction/example_test.csv'
EXAMPLE_SUBMISSION = '../input/ubiquant-market-prediction/example_sample_submission.csv'

In [ ]:
example_submission = pd.read_csv(EXAMPLE_SUBMISSION)
example_submission.head()

In [ ]:
example_test = pd.read_csv(EXAMPLE_TEST)
example_test.head()

In [ ]:
%%time

train_df = pd.read_feather(TRAIN)
train_df.head()

In [ ]:
split = RandomSplitter(valid_pct=0.1)(range_of(train_df))

cont = [f'f_{i}' for i in range(300)]

to = TabularPandas(train_df,
                   y_names='target',
                   cont_names=cont,
                   splits=split
                   )

dls = to.dataloaders(bs=4096)

In [ ]:
dls.show_batch()

In [ ]:
learn = tabular_learner(dls, layers=[300, 100, 30])

learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, cbs=[SaveModelCallback(monitor='valid_loss', comp=np.less)])

In [ ]:
import gc

del train_df
gc.collect()

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    dl = learn.dls.test_dl(test_df)
    y, _ = learn.get_preds(dl=dl)
    sample_prediction_df['target'] = y  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions